In [1]:
from tqdm import tqdm
import json

In [2]:
data = []
with open('processed-iium-confession.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l['en']:
            data.append(l)

In [3]:
len(data)

333758

In [4]:
# !pip3 install transformers -U
from transformers import AutoTokenizer, AutoModel
import torch

In [5]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-en')

In [6]:
model = AutoModel.from_pretrained('BAAI/bge-large-en')
_ = model.cuda()

2023-09-15 14:02:51.420023: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-15 14:02:51.500838: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-15 14:02:52.086558: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-15 14:02:52.086604: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

In [51]:
!rm -rf iium-confession
!mkdir iium-confession

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
instruction = 'Represent this sentence for searching relevant passages: '

In [53]:
import os

for i in tqdm(range(len(data))):
    filename = os.path.join('iium-confession', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    try:
        encoded_input = tokenizer([instruction + data[i]['en']],
                             return_tensors='pt')
        for k in encoded_input:
            encoded_input[k] = encoded_input[k].cuda()

        model_output = model(**encoded_input)
        sentence_embeddings = model_output[0][:, 0]
        sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
        v = sentence_embeddings[0].detach().cpu().numpy().tolist()
        d = {
            'text': data[i],
            'v': v,
        }
        with open(filename, 'w') as fopen:
            fopen.write(f'{json.dumps(d)}\n')
    except:
        pass

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 333758/333758 [1:02:42<00:00, 88.70it/s]


In [54]:
!du -hs iium-confession

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
7.8G	iium-confession
